In [1]:
import chromadb
from chromadb.utils import embedding_functions
import os
from dotenv import load_dotenv
import json

In [2]:
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
embedding_func  = embedding_functions.GoogleGenerativeAiEmbeddingFunction(api_key=api_key)

C:\Users\arind\Documents\LargeLanguageModels\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
CHROMA_PATH = "car_review_embeddings"
COLLECTION_NAME = "review_db"

In [5]:
#embedding_func = embedding_functions.SentenceTransformerEmbeddingFunction(model_name=EMBED_MODEL)
client = chromadb.PersistentClient(CHROMA_PATH)

try:
    client.delete_collection(name=COLLECTION_NAME)
except ValueError:
    print("Collection does not exist. Creating Now")

collection = client.create_collection(name=COLLECTION_NAME,
                                      embedding_function=embedding_func,
                                      metadata={"hnsw:space": "cosine"},
                                      )


Collection does not exist. Creating Now


In [6]:
with open('archive/car_reviews.json', 'r') as openfile:
 
    # Reading from json file
    data_dict = json.load(openfile)

In [7]:
ids = list(data_dict.keys())
reviews = [data_dict[id]['review'] for id in ids]
metadata = [data_dict[id]['metadata'] for id in ids]

In [8]:
collection.add(
    ids=ids,
    documents=reviews,
    metadatas=metadata,
)

In [9]:
question = """
What's the key to great customer satisfaction
based on detailed positive reviews?
"""

good_reviews = collection.query(query_texts=[question],
n_results=10,
include=["documents"],
where={"Rating": {"$gte": 3}},
)

In [10]:
good_reviews['documents']

[[' Very  pleased! Great deal! So far so good!',
  ' its fun to drive',
  ' Best power for the price. And it looks great.',
  " All shoppers out there, look for the best value for your dollar, don't allow unnecessary upgrades, extended warranties, you can get these items later on your own. Take time to decide what you want.",
  ' Car is awesome!!!',
  ' Pure fun to drive need to drive more',
  ' Great practical vehicle.  Love the stow and go storage under the floor and radio with hard drive for my music.  Has front and rear heated seats and USB ports.  Kids love it!',
  ' If you purchase this car you want regreted.',
  ' gt package is the way to go,,,plenty of power, good fuel economy',
  ' This is the most fun I ever had driving a car........I wish they made it when I was younger !']]

In [11]:
reviews_str = ",".join(good_reviews["documents"][0])

In [13]:
#TODO
'''
good_review_summaries = openai.ChatCompletion.create(
                            model="gpt-3.5-turbo",
                            messages=[
                            {"role": "system", "content": context.format(reviews_str)},
                            {"role": "user", "content": question},
                            ],
                            temperature=0,
                            n=1,
                            )
'''

'\ngood_review_summaries = openai.ChatCompletion.create(\n                            model="gpt-3.5-turbo",\n                            messages=[\n                            {"role": "system", "content": context.format(reviews_str)},\n                            {"role": "user", "content": question},\n                            ],\n                            temperature=0,\n                            n=1,\n                            )\n'